# Initialization

In [3]:
import numpy as np
import os
import PIL
import cv2
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import torch as t
import torchvision
import torchvision.transforms as transforms
import sys
import time
import torch.nn as nn
import torch.nn.functional as F
import imageio as iio
from skimage import filters
from skimage.color import rgb2gray  # only needed for incorrectly saved images
from skimage.measure import regionprops
from skimage.feature import canny
from scipy import ndimage as ndi
import shutil
#from models.res_gru_net import ResidualGRUNet 

## Default parameters 

In [4]:
ht = 1000
wt = 1800
depth = 2**8-1.
nh = 500
nw = 900

## Parameters depended on dataset 

In [5]:
#src = r'D:\data\599'.replace('\\', '/')
src = r'D:\seva\598_processing'.replace('\\', '/')
ido = '598'
idn = ido + 'prc'
dst = src.replace(ido,idn)
k = 90 #how many csv files in a folder, maximum is 90
C_in = 3 #how many views of one seed
D_out = 441

In [6]:
if t.cuda.is_available():
    device = "cuda"
else: 
    device = "cpu"
print(device)

cuda


## Timelapse printing

In [11]:
def ntime(s):
    m = 60
    l = [(s/m**2)%24, (s/m)%m, (s)%m]
    return([int(x) for x in l])
def coco(tl, ni):
    #s = sum(tl)
    l = tuple(ntime(np.mean(tl)*ni-sum(tl)))
    if len(tl)%10 != 9:
        print ("%02d:%02d:%02d," % l, end = '')
    else:
        print ("%02d:%02d:%02d" % l, end = '\n')

In [12]:
coco([200], 2000)

15:03:20,

In [13]:
print('%02d' % 1)

01


In [45]:
np.mean([1,2,3]).dtype

dtype('float64')

In [42]:
ntime(500)

[0, 8, 20]

# Remove background and crop relatively to center of mass 

In [61]:
def preproc(ci):
    image = depth - iio.imread(ci)
#     image[image<30] = 0
#     image[image>200] = 1
    threshold_value = filters.threshold_otsu(image)
    fmask = (image > threshold_value)
    nimage = fmask*image
#     qmask = ndi.binary_fill_holes(canny(timage/depth,sigma=sg)+frame)
#     nimage = (1-frame)*qmask*image
    properties = regionprops(fmask.astype(int), nimage)
    cms = tuple(map(lambda x: int(x), properties[0].centroid))
    pimage = (depth - nimage[cms[0]-nh//2:cms[0]+nh//2,cms[1]-nw//2:cms[1]+nw//2]).astype(np.uint8)
    #print('.', end = '')
    return(pimage)

In [14]:
def ig_f(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir, f))]

shutil.copytree(src, dst, ignore=ig_f)
#shutil.copytree(src, dst)

'D:/seva/598prc_processing'

In [15]:
frame = np.zeros([ht,wt])
frame[ht-1][:] = 1

In [ ]:
ds = []
tl = []
i = 0
numim = len([x[0] for x in os.walk(src)])
for root, directories, filenames in os.walk(src):
    start = time.time()
    #print(directories)
    if ds == []:
        ds = directories
    for filename in filenames:  
        fp = os.path.join(root,filename)
        if filename[-5:] == '0.tif':
            ci = os.path.join(root,filename)
            #print(ci)
# #             sg = 3.5
#             ii = 0
# #             #ni = preproc(ci,printg)
#             #tf = 5
#             sg = 4
#             while ii < 1e6: 
#                 #tf += 10
#                 ni = preproc(ci,sg)
#                 ii = np.sum(depth - ni)
#                 sg += 1
            #print(sg)
            #print(np.sum(depth - ni))
            #ni = preproc(ci)
            ni = preproc(ci)
            ci = ci.replace('/598_','/598prc_')
            PIL.Image.fromarray(ni).save(ci[:-4]+'prc.tif')
            #print('.', end = '')

        elif filename[-7:] == '_20.csv':
            shutil.copyfile(os.path.join(root,filename),os.path.join(root,filename).replace('/598_','/598prc_'))
        fn = time.time()
    i+=1
    end = time.time()
    tl.append(end-start)
    #print(tl)
    coco(tl,numim)
    #print('|', end = '')
#PIL.Image.fromarray(ni).show()

00:00:00,00:01:48,00:06:09,00:08:28,00:09:50,00:10:59,00:11:28,00:12:11,00:12:32,
00:12:48,00:13:08,00:13:19,00:13:31,00:13:43,00:13:46,00:13:52,00:13:55,00:14:06,00:14:09,
00:14:17,00:14:17,00:14:21,00:14:24,00:14:30,00:14:32,00:14:38,00:14:37,00:14:39,00:14:40,
00:14:44,00:14:49,00:14:53,00:14:56,00:14:57,00:14:55,00:14:55,00:14:56,00:15:00,00:15:01,
00:15:01,00:15:00,00:15:03,00:15:04,00:15:10,00:15:20,00:15:21,00:15:23,00:15:24,00:15:26,
00:15:32,00:15:31,00:15:32,00:15:31,00:15:29,00:15:29,00:15:29,00:15:35,00:15:36,00:15:38,
00:15:42,00:15:42,00:15:46,00:15:47,00:15:47,00:15:48,00:15:46,00:15:45,00:15:45,00:15:47,
00:15:47,00:15:46,00:15:46,00:15:45,00:15:45,00:15:45,00:15:46,00:15:45,00:15:44,00:15:43,
00:15:42,00:15:42,00:15:43,00:15:43,00:15:44,00:15:43,00:15:42,00:15:41,00:15:40,00:15:41,
00:15:41,00:15:40,00:15:40,00:15:39,00:15:39,00:15:39,00:15:40,00:15:39,00:15:38,00:15:37,
00:15:36,00:15:35,00:15:37,00:15:37,00:15:37,00:15:35,00:15:35,00:15:35,00:15:36,00:15:37,
00:15:38

In [49]:
tl = [0.0, 0.03490328788757324, 0.16156506538391113]

In [58]:
#coco(tl, ni)
print(ni)
print(np.mean(tl)*ni)
#print(np.mean(tl)*ni-sum(tl))
#ntime(np.mean(tl)*ni-sum(tl))

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[[25.04971504 25.04971504 25.04971504 ... 25.04971504 25.04971504
  25.04971504]
 [25.04971504 25.04971504 25.04971504 ... 25.04971504 25.04971504
  25.04971504]
 [25.04971504 25.04971504 25.04971504 ... 25.04971504 25.04971504
  25.04971504]
 ...
 [25.04971504 25.04971504 25.04971504 ... 25.04971504 25.04971504
  25.04971504]
 [25.04971504 25.04971504 25.04971504 ... 25.04971504 25.04971504
  25.04971504]
 [25.04971504 25.04971504 25.04971504 ... 25.04971504 25.04971504
  25.04971504]]


In [10]:
path = dst
ds = [f.path for f in os.scandir(path) if f.is_dir()]
print(ds)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/seva/598prc_processing'

## Subtract trapezoid 

In [ ]:
#for x in range(wt):
xshift = int(np.mean([695,685,720,695,700,693,685]))
tr = np.zeros([ht,wt])
a, b, c = np.mean([173,174,177]), np.mean([92,92,91]), np.mean([310,306,319,308,308,314])
h = np.sqrt(c**2-((a-b)/2)**2)
for x in range(int(h)-20):
    tr[x+xshift,int((a-b)/c/2*(h-x))+yshift:int(x*(a-b)/2/h+(a+b)/2)+yshift] = 255
tr = depth - tr
PIL.Image.fromarray(tr).show()
PIL.Image.fromarray(image*tr/depth).show()
print(int((a-b)/c/2*(h-y)),int((a-b)/c/2*(h-y)))

# Training

## Image to tensor 

In [14]:
#print(ds)
#x = os.walk(rootdir)
#[q, w, e] = os.walk(rootdir)
# k = 0
# for root, directories, filenames in os.walk(rootdir): 
#     for filename in filenames:  
#         if filename[-3:] == 'csv':
#             k += 1
def getbatch(cds):
    data = t.zeros([k, C_in, nh, nw]).to(device)
    label = t.zeros([k, D_out]).to(device)
    c, dc = [0]*2
    for root, directories, filenames in os.walk(cds): 
        #for directory in directories:
        i = 0
        lc = 0
        for filename in filenames:
            ci = os.path.join(root,filename)
            if filename[-3:] == 'csv':
                #print(ci,lc)
                label[lc,:] = t.tensor(np.genfromtxt(ci, delimiter='\n'))
                lc += 1
            if filename[-3:] == 'tif':
                #print(ci,dc)
                data[dc, i, :, :] = transforms.ToTensor()(cv2.imread(ci, 0)).unsqueeze_(0)
                dc += i//2
                i += 1
    data = data/255
    #label = label 
    return(data, label)

In [15]:
data, label = getbatch(os.path.join(dst,'0'))

In [25]:
print("%.2f", % 3.1414)

SyntaxError: invalid syntax (<ipython-input-25-86a3bac8786f>, line 1)

In [22]:
#data = t.zeros([k, 3, 1000, 1800])
print("%.2f", %(data.element_size() * data.nelement()/1024**3), 'GB')
#print(label.element_size() * label.nelement()/1024)

SyntaxError: invalid syntax (<ipython-input-22-960ce97e3d13>, line 2)

## Neural network architecture

In [105]:
class DNet(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = t.nn.Linear(nh*nw*C_in, layers[0])
        self.linear2 = t.nn.Linear(layers[0],layers[1])
        self.linear3 = t.nn.Linear(layers[1],layers[2])
        self.linear4 = t.nn.Linear(layers[2], D_out)
        self.do = t.nn.Dropout(p=dop)
    def forward(self, x):
        #print(x.shape)
        x = x.view(x.shape[0],-1)
        #print('x flattened', x.shape)
        #print(nh,nw,C_in, layers[0])
        x = self.do(F.leaky_relu(self.linear1(x).clamp(min=0)))
        #print(x.shape)
        x = self.do(F.leaky_relu(self.linear2(x)))
        x = self.do(F.relu(self.linear3(x)))
        x = self.do(self.linear4(x))
        return x
class CNet(t.nn.Module):
    def __init__(self):
        super().__init__()
#        super(CNet, self).__init__()
        self.conv0 = nn.Conv2d(C_in, 6, 5)
        self.conv1 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear0 = t.nn.Linear(16*47*47, 1000)
        self.linear1 = t.nn.Linear(1000, D_out)

    def forward(self, x):
        x = self.pool(F.relu(self.conv0(x)))
        print(x.shape)
        #print(t.mean(x))
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, x.size(1)*x.size(2)*x.size(3))
        x = F.relu(self.linear0(x))
        x = F.relu(self.linear1(x))
        #x = self.linear1(x)
        return x

In [106]:
nnarchitectures = { 'DNet' : DNet, 
                    'CNet' : CNet}

In [111]:
lossa, timelapse = multit()

..........0:0:0,

## Training algorithm

In [136]:
def multit():
    starta = time.time()
#     model = None
#     if model is None:
    model = nnarchitectures[modelname]()
    model.to(device)
    optimizer = t.optim.Adam(model.parameters(), lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    tl = []
    lossa = np.zeros([2, epochs])
    #xval = np.zeros([])
    xval = t.zeros([k, C_in, nh, nw]).to(device)
    yval = t.zeros([k, D_out]).to(device)
    [xval, yval] = getbatch(ds[1])
    for j in range(epochs):
        start = time.time()  
        for i in range(bn):
            #print('.', end = '')
            #data = idata[i]
            [xtrain, ytrain] = getbatch(ds[i+1])
            loss = []
            criterion = t.nn.MSELoss(reduction='mean')
            #start = time.time()
            #print(tm)
            xtrain = xtrain.detach() #xtrain contains images of the circle
            y_pred = model(xtrain) #y_pred consists of coordinates of the circle center and its radius
            #print(y_pred.shape)
           #print(ytrain.shape)
            if ltype == 'supervised': 
                # Forward pass: Compute predicted y by passing x to the model
                #calculating loss between predicted parameters and parameters which were used for generation
                loss = criterion(y_pred, ytrain)
            else: 
                #x_pred = t.zeros([N,1,sz,sz])
                x_pred = t.zeros([bn,C_in,ht,wt])
                for i in range(bn):
                    # Generating circles with inferred parameters y_pred.
                    #print(y_pred)
                    x_pred[i,0,:,:] = ellipses(y_pred, k, sz, device)
                # Compute and print loss
                # Compare generated image with the input image, finding per-pixel distance between two images
                loss = criterion(x_pred, xtrain)
                #print(loss.item())
            #print('trainig loss counter', i+j*bn)
            #lossa[0][i+j*bn] = loss.item() #the first number in the loss is a training loss
            #updating list with losses 
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #lossc = lossa
            #lossa[1][i+j*bn] = criterion(model(xval), yval).item() #the second number in the loss is a validation loss
            #print(i+j*bn)
#             end = time.time()
#             tl.append(end-start)
#             ntime(sum(tl) / len(tl)*bn - sum(tl))
        lossa[0][j] = loss.item()
        lossa[1][j] = criterion(model(xval), yval).item()
        end = time.time()
        tl.append(end-start)
        coco(tl, i+j, epochs)
#         l = ntime(sum(tl)/len(tl)*epochs-sum(tl))
#         if j%10 != 9:
#             print ("%d:%d:%d," % tuple(l), end = '')
#         else:
#             print ("%d:%d:%d" % tuple(l), end = '\n')
    enda = time.time()
    timelapse = enda - starta
    return(lossa, timelapse)

In [137]:
lossa, timelapse = multit()

0:1:28,0:1:4,0:0:48,0:0:36,0:0:27,0:0:20,0:0:14,0:0:9,0:0:4,0:0:0


In [134]:
(l[0], l[1], l[2])

(0, 8, 20)

In [135]:
l = ntime(500)
print ("%d:%d:%d," % tuple(l), end = '')   

0:8:20,

## Draw a plot of the training curve

In [39]:
def traplot():
    plt.rcParams["figure.figsize"] = (18,10)
    #linestyles = ['-', '--', '-.', ':', '-', '--', '-.', ':', '-', '--', '-.', ':']
    colors = ['red', 'green']
    axes = plt.gca()
    #axes.set_ylim([0,0.02])
    labels_text = ['training', 'validation']
    #print(lossl.shape)
    #print(np.arange(epochs*bn).shape)
    for trt in range(len(labels_text)):
        plt.plot(np.arange(lossa.shape[1]), lossa[trt][:], label=labels_text[trt], \
                 color = colors[trt], linewidth=3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='best', borderaxespad=0., fontsize = 24)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.autoscale(enable=True, axis='both', tight=None)
    plt.xlabel('Epoch', fontsize=24)
    plt.ylabel('MSE Loss', fontsize=24)
    #+n+bn+bs+epochs+lr+imgsize+layers+dop+
    plt.savefig('k'+str(k)+'_'+'n'+str(n)+'_'+'_'+'bn'+str(bn)+','+'bs'+str(bs)+'_'+\
                'epochs'+str(epochs)+'_'+'layers'+str(layers)+'_'+'lr'+str(lr)+'_'+'etime'+\
                format(timelapse,".2f") + '_' + 'noisein' + str(randinit) +'.png', bbox_inches='tight')
    plt.show()

## Hyperparameters 

In [11]:
k = 3

In [95]:
print(dst)

D:/data/599prc


In [102]:
epochs = 10 #how many epochs 
lr = 5e-5 #learning rate
bn = 1 #how many batches
bs = 3 #how many images in a batch
layers = [100, 50, 441] #number of neurons at every layer
dop = 0 #drop out percentage 0 to 1. 
randinit = 0 #0 is ellipse generator, 1 is noise generator
modelname = 'DNet'
ltype = 'supervised'

In [65]:
lossa, timelapse = multit()

torch.Size([3, 3, 500, 900])
torch.Size([3, 1350000])


RuntimeError: The size of tensor a (1350000) must match the size of tensor b (441) at non-singleton dimension 1

In [ ]:
plt.rcParams["figure.figsize"] = (18,10)
nrows, ncols =1,4
fig = plt.figure()
for i in range(nrows*ncols):
        ax = fig.add_subplot(int(str(nrows)+str(ncols)+str(i+1)), projection='3d') 
        ax.voxels(datac[i+3][0])
        plt.axis('off')
        plt.autoscale(enable=True, axis='both', tight=None)
#dt = str(datetime.fromtimestamp(int(time.time())))
plt.savefig(os.getcwd() + '\plot output'+'\k'+str(k)+'_'+'n'+str(n)+'_' +\
            'time' + time.strftime("%H%M%S") + '.png', bbox_inches='tight')